<a href="https://colab.research.google.com/github/kevin6449/ironman2024_genai/blob/main/gen_text_day7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 瞭解 Gemini API 的文件處理功能

Gemini API 可處理傳入的 PDF 文件，並進行推論。時間 上傳 PDF 檔案後，Gemini API 就能：

說明或回答內容相關問題
提供內容的摘要
從內容推斷


In [1]:
import google.generativeai as genai

In [2]:
from google.colab import userdata

API_KEY=userdata.get('GOOGLE_API_KEY')

In [3]:
#genai.configure(api_key="YOUR_API_KEY")

# Configure the client library by providing your API key.
genai.configure(api_key=API_KEY)

## 使用 File API 上傳文件
使用 File API 上傳任何大小的文件。( 想要傳送的檔案與系統指示組合較大 大於 20 MB)。

In [4]:
!curl -o gemini.pdf https://storage.googleapis.com/cloud-samples-data/generative-ai/pdf/2403.05530.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7059k  100 7059k    0     0  16.9M      0 --:--:-- --:--:-- --:--:-- 16.9M


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

使用 media.upload 和 輸出 URI，此 URI 在 Gemini API 呼叫中會用做參考：




In [5]:
# Upload the file and print a confirmation
sample_file = genai.upload_file(path="gemini.pdf",
                                display_name="Gemini 1.5 PDF")

print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.uri}")


Uploaded file 'Gemini 1.5 PDF' as: https://generativelanguage.googleapis.com/v1beta/files/q5sm8ac67ph8


# 驗證 PDF 檔案上傳及取得中繼資料
您可以驗證 API 是否已成功儲存上傳的檔案，並取得其 透過 SDK 呼叫 files.get 來更新中繼資料僅限 name (及 副檔名為 uri) 重複。只有在符合以下條件時，才使用「display_name」來識別檔案 以及自己的特色




In [6]:
file = genai.get_file(name=sample_file.name)
print(f"Retrieved file '{file.display_name}' as: {sample_file.uri}")

Retrieved file 'Gemini 1.5 PDF' as: https://generativelanguage.googleapis.com/v1beta/files/q5sm8ac67ph8


# 使用上傳的文件，提示 Gemini API
上傳檔案後，您可以提出參照的 GenerateContent 要求 File API URI選取生成式模型，並以文字提示提供模型 以及上傳的文件：




In [7]:
# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

# Prompt the model with text and the previously uploaded image.
response = model.generate_content([sample_file, "您能否將這份文件總結為項目符號清單？"])

print(response.text)

當然可以。以下是文件的摘要：

* **簡介**。這篇文章介紹了 Gemini 1.5 Pro，這是 Google DeepMind 所開發的一種新的多模態混合專家模型，能夠處理超過 1000 萬個標記的上下文。
* **模型架構**。Gemini 1.5 Pro 是基於稀疏混合專家 (MoE) 轉換器的，並建立在 Google 在 MoE 模型和語言模型研究上的過去工作之上。
* **訓練基礎架構和數據集**。Gemini 1.5 Pro 訓練於多個 4096 芯片的 Google TPUv4 加速器，這些加速器分佈在多個數據中心，並使用多模態和多語言數據。
* **長上下文評估**。由於模型的長上下文能力，研究人員對模型進行了大型定量評估，包括針對長序列的困惑度，以及「乾草堆中的針」任務。
* **現實長上下文評估**。研究人員通過新的任務來評估模型，這些任務需要更長更真實的上下文，例如從單本書中學習翻譯新語言。
* **核心能力評估**。研究人員評估了模型在沒有長上下文設置的情況下在核心能力方面的表現，例如數學、科學和推理，編碼，多語言，以及指令遵循。
* **負責任的部署**。研究人員遵循一個結構化的方法，在 Gemini 模型的部署中考慮倫理和安全問題。
* **討論**。文章討論了長上下文評估的挑戰，以及在未來發展中使用更先進的長上下文模型的重要性。

希望這個摘要有幫助！


# 上傳一或多個儲存在本機的檔案
或者，您也可以上傳一或多個本機儲存的檔案。

如果您打算傳送的檔案和系統指示組合， 如果超過 20 MB，請使用 File API 上傳這些檔案 。使用者可透過 Gemini API 在本機呼叫較小的檔案：

In [11]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.5 MB/s eta 0:00:00


In [14]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        extracted_text = ""
        for page in pdf_reader.pages:
            text = page.extract_text()
            if text:
                extracted_text += text
        return extracted_text

sample_file_2 = extract_text_from_pdf('/content/drive/MyDrive/OWASP_Top_10-2017_(en).pdf.pdf')
#sample_file_3 = extract_text_from_pdf('example-2.pdf')

# 顯示多份文件的提示
您與 Gemini API 的任何文件和文字組合都可以用於 會套用在模型的背景區間內這個範例提供一則簡短文字 提示和先前上傳的n份文件：

In [18]:
# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

prompt = "總結這些文件的論文陳述之間的差異。,  請用繁體中文回答"

response = model.generate_content([prompt, sample_file, sample_file_2])

print(response.text)

本文档介绍了 Gemini 1.5 Pro，这是 Gemini 1.5 系列中的第一个模型。与 Gemini 1.0 系列模型相比，Gemini 1.5 Pro 在计算效率、多模态能力、长文本推理和下游性能方面有了显著提升。

本文档主要阐述了 Gemini 1.5 Pro 在以下方面的优势：

**1.  长文本理解能力**：Gemini 1.5 Pro 将内容窗口扩展到了数百万个标记，远超 Gemini 1.0 系列模型的 32k 个标记。这使得 Gemini 1.5 Pro 能够处理比以往更长的文本，如多篇文档、视频和音频，并能够在这些长文本中进行推理和检索。

**2.  多模态能力**：Gemini 1.5 Pro 是一个多模态模型，能够理解和处理文本、视频和音频。这使得 Gemini 1.5 Pro 能够在现实世界中执行更复杂的推理任务，例如长文本问答、长视频问答和长文本自动语音识别。

**3.  高效性**：与 Gemini 1.0 Ultra 相比，Gemini 1.5 Pro 使用了更少的训练计算资源，但仍然在大部分基准测试中表现出色。

**4.  在文本领域表现出色**：Gemini 1.5 Pro 在文本领域的表现也十分出色，在数学、科学和推理、编码、多语言和指令遵循方面均超越了 Gemini 1.0 Pro 和 Gemini 1.0 Ultra。

**5.  在视觉领域取得了改进**：Gemini 1.5 Pro 在视觉领域也取得了改进，在图像理解和视频理解方面超越了 Gemini 1.0 Pro。

**6.  在音频领域取得了显著进展**：在音频领域，Gemini 1.5 Pro 能够理解和处理比以往更长的音频片段。

**7.  负责任的部署**：本文档还介绍了 Gemini 1.5 Pro 的负责任部署策略，包括对模型的影响评估、安全评估和缓解措施。

总的来说，Gemini 1.5 Pro 是一个非常强大的模型，它在多模态能力、长文本理解能力和效率方面都取得了重大突破。Gemini 1.5 Pro 的发布标志着大型语言模型领域取得了重要进展，也为未来 AI 模型的发展方向提供了新的思路。



# 可列出檔案
您可以使用 File API 列出透過 File API 上傳的所有檔案及其 URI files.list_files():

In [ ]:
# List all files
for file in genai.list_files():
    print(f"{file.display_name}, URI: {file.uri}")

# 刪除檔案
使用 File API 上傳的檔案會在 2 天後自動刪除。個人中心 您也可以使用 files.delete_file() 手動刪除：




In [ ]:
# Delete file
genai.delete_file(document_file.name)
print(f'Deleted file {document_file.uri}')